In [65]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range
import scipy.io as spio

Import Data from Matlab

In [66]:
mat = spio.loadmat('../ReflectionModel/dataset_angle_simple.mat')
features = mat['features']
labels = mat['labels']
print(len(features))
print(len(features[0]))

100000
186


Preprocess Data to make it 0-mean, 1 variance

In [67]:
feature_mean = np.mean(features,axis=0)
label_mean = np.mean(labels,axis=0)
label_std = np.std(labels,axis=0)
feature_std = np.std(features,axis=0)
print(feature_std.shape)
feature_mean_m = np.tile(feature_mean, [len(features),1])
feature_std_m = np.tile(feature_std, [len(features),1])
label_mean_m = np.tile(label_mean,[len(labels),1])
label_std_m = np.tile(label_std,[len(labels),1])
print(feature_std_m.shape)
features_normal = np.divide(features - feature_mean_m, feature_std_m)
labels_normal = np.divide(labels- label_mean_m, label_std_m)
print(features_normal.shape)
print(labels_normal.shape)
print(label_std)
print(feature_std)

(186,)
(100000, 186)
(100000, 186)
(100000, 1)
[ 0.76299285]
[ 2.13885275  2.12586838  2.11913876  2.112037    2.10288093  2.09555447
  2.08996094  2.08706944  2.08379891  2.07853738  2.07279584  2.06910597
  2.06775207  2.06388206  2.05790614  2.05680917  2.05478722  2.05082704
  2.04846414  2.04444328  2.0396728   2.03278749  2.02767514  2.02123701
  2.01924829  2.01533707  2.01171056  2.00839438  2.00073642  1.99245511
  1.98750522  2.19434153  2.18236285  2.16909533  2.15698701  2.14764895
  2.13388117  2.12405426  2.11431177  2.10784453  2.10130708  2.09162945
  2.0795149   2.06860628  2.05490504  2.03912722  2.02548957  2.01226039
  1.99936585  1.98677003  1.97416822  1.96021765  1.94736174  1.93514418
  1.92510929  1.91220902  1.90244702  1.89393122  1.88494358  1.87339149
  1.86132989  1.85025253  2.5108094   2.49401267  2.47814498  2.46367394
  2.4494279   2.43750271  2.42160343  2.40791853  2.39654557  2.38574892
  2.37260238  2.36128323  2.34734086  2.33553807  2.32488468  2

In [68]:
train_pts = 60000
valid_pts = 20000
test_pts = 20000
print(type(features_normal[0][0]))
features_normal = features_normal.astype('float32')
print(type(features_normal[0][0]))
labels_normal = labels_normal.astype('float32')
train_dataset = (features_normal[0:train_pts ,:])
train_labels = labels_normal[0:train_pts,:]
valid_dataset = features_normal[train_pts:valid_pts+train_pts,:]
valid_labels = labels_normal[train_pts:valid_pts+train_pts,:]
test_dataset = features_normal[train_pts+valid_pts:train_pts+valid_pts+test_pts,:]
test_labels = labels_normal[train_pts+valid_pts:train_pts+valid_pts+test_pts,:]
print(train_dataset.shape)
print(train_dataset[0:10,0:10])

<type 'numpy.float64'>
<type 'numpy.float32'>
(60000, 186)
[[ 0.41808215 -0.47943574  0.38616794 -0.21805754 -0.01301382  0.23585451
  -0.40407693  0.47804517 -0.4022705   0.22515273]
 [ 0.42665854 -0.30114302  0.04161127  0.21832512 -0.39560214  0.47276425
  -0.40188807  0.22286989  0.02494384 -0.2869508 ]
 [ 0.11848643 -0.4462274   0.50638551 -0.2792367  -0.13937524  0.45667034
  -0.47120163  0.21638024  0.17445189 -0.49192542]
 [ 0.30815765 -0.52816898  0.41036683 -0.05713986 -0.35573241  0.5247069
  -0.36773649 -0.01014095  0.36529934 -0.49651036]
 [ 0.55479205 -0.33839005 -0.21932279  0.56438237 -0.3991901  -0.14007214
   0.52588093 -0.44491288 -0.0550025   0.49516603]
 [ 0.38972387 -0.39739358  0.3883346  -0.34669426  0.38122916 -0.37023512
   0.37308007 -0.34772313  0.34754252 -0.34497473]
 [ 0.06836407 -0.06474066  0.07587463 -0.09786237  0.08641557 -0.10447545
   0.12427203 -0.13410933  0.12846819 -0.13325036]
 [ 0.19032146 -0.35056889  0.41219044 -0.45743868  0.39379415 -0.25

Setup TensorFlow graph

In [74]:
batch_size = 2048
num_hidden = [1024,512,256]
depth = 4
graph = tf.Graph()
keep_prob = 0.9
decay_rate = 0.5
decay_steps = 1000
num_features =features.shape[1]
num_labels = labels.shape[1]
print(num_features)
def get_logits_for_training(dataset,w,b):
    cur_mat = dataset
    for i in range(0,depth-1):
        cur_mat = tf.matmul(cur_mat,w[i])+b[i]
        cur_mat = tf.nn.dropout(tf.nn.relu(cur_mat),keep_prob)
    logits = tf.matmul(cur_mat,w[depth-1])+b[depth-1]
    return logits

def get_logits_for_prediction(dataset,w,b):
    cur_mat = dataset
    for i in range(0,depth-1):
        cur_mat = tf.matmul(cur_mat,w[i])+b[i]
        cur_mat = tf.nn.relu(cur_mat)
    logits = tf.matmul(cur_mat,w[depth-1])+b[depth-1]
    return logits
    
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, num_features))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  w = []
  b = []
  w.append(tf.Variable(tf.truncated_normal([num_features, num_hidden[0]],stddev=0.01)))
  b.append(tf.zeros([num_hidden[0]]))
  for i in range(1,depth-1):
    w.append(tf.Variable(tf.truncated_normal([num_hidden[i-1],num_hidden[i]], stddev=0.01)))
    b.append(tf.zeros([num_hidden[i]]))
  w.append(tf.Variable(tf.truncated_normal([num_hidden[depth-2], num_labels],stddev=0.01)))
  b.append(tf.Variable(tf.zeros([num_labels])))
  
  # Training computation.  
  logits = get_logits_for_training(tf_train_dataset,w,b)
  #loss = tf.reduce_mean(
  #  tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
  loss = tf.sqrt(tf.reduce_mean(tf.squared_difference(tf_train_labels, logits)))
  #Optimizer
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, decay_steps, decay_rate)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  logits_predict = get_logits_for_prediction(tf_train_dataset,w,b)
  train_prediction = (logits_predict)
  valid_prediction = (get_logits_for_prediction(tf_valid_dataset,w,b))
  test_prediction = (get_logits_for_prediction(tf_test_dataset,w,b))

186


In [75]:
def accuracy(predictions, labels):
  return np.sqrt(((predictions - labels) ** 2).mean())

def accuracy_xy(predictions,labels):
    return (np.sqrt(((predictions[:,0] - labels[:,0]) ** 2).mean()),np.sqrt(((predictions[:,1] - labels[:,1]) ** 2).mean()))
    
num_steps = 10002
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions,l1 = session.run(
      [optimizer, loss, train_prediction,w], feed_dict=feed_dict)
    if (step % 100 == 1):
      print("Minibatch loss at step %d: %f" % (step, l))
      print(l1[1].shape)
        
      print("Minibatch accuracy: %.3f" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.3f" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("Test accuracy: %.10f" % accuracy(test_prediction.eval(), test_labels))
     # print(accuracy_xy(test_prediction.eval(),test_labels))

Initialized
Minibatch loss at step 1: 0.998284
(1024, 512)
Minibatch accuracy: 0.998
Validation accuracy: 0.996
Test accuracy: 0.9994797707
Minibatch loss at step 101: 0.996916
(1024, 512)
Minibatch accuracy: 0.997
Validation accuracy: 0.996
Test accuracy: 0.9993438721
Minibatch loss at step 201: 0.997190
(1024, 512)
Minibatch accuracy: 0.997
Validation accuracy: 0.995
Test accuracy: 0.9985618591
Minibatch loss at step 301: 0.923328
(1024, 512)
Minibatch accuracy: 0.924
Validation accuracy: 0.891
Test accuracy: 0.9057578444
Minibatch loss at step 401: 0.471262
(1024, 512)
Minibatch accuracy: 0.459
Validation accuracy: 0.558
Test accuracy: 0.5536177754
Minibatch loss at step 501: 1.003580
(1024, 512)
Minibatch accuracy: 1.004
Validation accuracy: 0.996
Test accuracy: 0.9994711280
Minibatch loss at step 601: 1.016550
(1024, 512)
Minibatch accuracy: 1.017
Validation accuracy: 0.996
Test accuracy: 0.9994658828
Minibatch loss at step 701: 1.004637
(1024, 512)
Minibatch accuracy: 1.004
Valid

Validation accuracy: 0.996
Test accuracy: 0.9992073178
Minibatch loss at step 6301: 1.012051
(1024, 512)
Minibatch accuracy: 1.012
Validation accuracy: 0.992
Test accuracy: 0.9945356846
Minibatch loss at step 6401: 0.983258
(1024, 512)
Minibatch accuracy: 0.984
Validation accuracy: 0.989
Test accuracy: 0.9919553399
Minibatch loss at step 6501: 0.994064
(1024, 512)
Minibatch accuracy: 0.994
Validation accuracy: 0.988
Test accuracy: 0.9904864430
Minibatch loss at step 6601: 1.001025
(1024, 512)
Minibatch accuracy: 1.001
Validation accuracy: 0.996
Test accuracy: 0.9991341829
Minibatch loss at step 6701: 0.968625
(1024, 512)
Minibatch accuracy: 0.969
Validation accuracy: 0.986
Test accuracy: 0.9882664084
Minibatch loss at step 6801: 0.992789
(1024, 512)
Minibatch accuracy: 0.993
Validation accuracy: 0.984
Test accuracy: 0.9863349795
Minibatch loss at step 6901: 1.008674
(1024, 512)
Minibatch accuracy: 1.009
Validation accuracy: 0.996
Test accuracy: 0.9995317459
Minibatch loss at step 7001: